In [90]:
import pandas as pd 
import numpy as np 
import re
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [207]:
funds = pd.read_csv("../../data/FundTick.csv")
#for testing and development purposes
funds = funds.iloc[1:100,]
funds

,Unnamed: 0,summary_period2,crsp_fundno,caldt,cusip8,crsp_portno,fund_name,ticker,ncusip,index_fund_flag,et_flag,not_index
1,1355350,Q,35897,2007-09-28,23337676,1004951.0,DWS Equity Trust: DWS Alternative Asset Alloca...,AAAAX,233376763,NaN,NaN,1
2,55011,Q,4118,2000-03-31,01852K20,1003334.0,"Alliance All-Asia Investment Fund, Inc.; Class...",AAABX,01852K201,NaN,NaN,1
3,55033,Q,4119,2000-03-31,01852K30,1003334.0,"Alliance All-Asia Investment Fund, Inc.; Class...",AAACX,01852K300,NaN,NaN,1
4,1891360,Q,54371,2012-03-30,02094081,1029342.0,Alpine Series Trust: Alpine Accelerating Divid...,AAADX,020940813,NaN,NaN,1
5,2249518,Q,97179,2020-09-30,01892033,1024646.0,Virtus Investment Trust: Virtus AllianzGI Heal...,AAAEX,018920330,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1776225,Q,49953,2010-09-30,00768Y20,1020722.0,AdvisorShares Trust: WCM/BNY Mellon Focused Gr...,AADR,00768Y206,NaN,F,0
96,13960,AQ,3069,2004-12-31,00141M51,1002543.0,AIM Growth Series: AIM Aggressive Allocation F...,AADRX,00141M515,NaN,NaN,1
97,1704210,AQ,47687,2009-12-31,00888W20,1002543.0,AIM Growth Series: AIM Growth Allocation Fund;...,AADSX,00888W205,NaN,NaN,1
98,113725,Q,5201,2007-03-30,02630T77,1004887.0,American Funds Target Date Retirement Series: ...,AADTX,02630T779,NaN,NaN,1


In [209]:
model_dict = {"sustainability_rating": None, 
                    "global_category_count": None, 
                    "sustainable_investment": None, 
                    "hist_sustainability_score": None, "current_sus_score": None, "hist_avg": None,
                    "environmental_rating": None, 
                    "social_rating": None, 
                    "governance_rating": None, 
                    "unallocated_rating": None, 
                    "carbon_current": None, "carbon_low": None, "carbon_high": None, "carbon_average": None, 
                    "fossil_current": None, "fossil_low": None, "fossil_high": None, "fossil_avg": None}

for key in model_dict.keys(): 
    funds[key] = None
    
for key in structured.keys(): 
    funds.loc[99, key] = structured[key]
    

In [221]:
re.findall("no", yes_no_regex)


['no']

In [217]:
index_link = "https://www.morningstar.com/etfs/arcx/{}/portfolio"
not_index_link = "https://www.morningstar.com/funds/xnas/{}/portfolio"
def generate_link(fund_name, is_not_index): 
    if is_not_index: 
        return not_index_link.format(fund_name)
    
    return index_link.format(fund_name)

funds["link"] = funds.apply(lambda row: generate_link(row.ticker.lower(), row.not_index), axis = 1)

In [189]:
class ESG_Parser: 
    
    def __init__(self): 
        self.driver = self.start_selenium()
        
    def parse_morningstar(self, weblink): 
        """
        Method that controls flow of work, and handles 
        errors. 
        """
        data = self.get_data_from_webpage(weblink)
        data_points = self.parse_data(data) 
        return data_points
        

    def start_selenium(self): 
        """
        Starts new selenium session, allowing for multiple web requests in a single session 
        saving startup time when parsing new website. 
        """
        driver = webdriver.Chrome("./chromedriver_win32/chromedriver.exe")
        return driver 
    
    def link_is_broken(self, weblink): 
        """
        Morningstar still accepts non-findable webpages without an error which means we must 
        manually check to see if a funds website exists. 
        """
        try: 
            self.driver.find_element(By.CLASS_NAME, "error")
            return True 
        except NoSuchElementException: 
            return False
        
    def get_data_from_webpage(self, weblink): 
        """
        Takes in a weblink for morningstar and extracts the ESG data from it. 
        Two methods are used finding all the "sr-only" tags which contain data for 
        this portion of the website, and loking at the sustainbility dp-value (only for
        number of funds in category). 
        """
        driver = self.driver

        driver.get(weblink)
        sleep(2) 

        if self.link_is_broken(weblink): 
            return None 

        info1 = []
        for item in driver.find_elements(By.CLASS_NAME, "sal-sustainability__dp-value"): 
            info1.append(item.text)

        info2 = []
        for item in driver.find_elements(By.CLASS_NAME,"sr-only"): 
            info2.append(item.text)

        return [info1, info2]
    
    def parse_data(self, data): 
        """
        Pass in data from the get_data_from_webpage() function, and if the 
        data exists a dictionary with the relevant information will be returned. 
        """
        esg_dict = {"sustainability_rating": None, 
                    "global_category_count": None, 
                    "sustainable_investment": None, 
                    "hist_sustainability_score": None, "current_sus_score": None, "hist_avg": None,
                    "environmental_rating": None, 
                    "social_rating": None, 
                    "governance_rating": None, 
                    "unallocated_rating": None, 
                    "carbon_current": None, "carbon_low": None, "carbon_high": None, "carbon_average": None, 
                    "fossil_current": None, "fossil_low": None, "fossil_high": None, "fossil_avg": None}
        
        #make sure data exists (morningstar doesn't list every fund)
        if not data: 
            return esg_dict 
        
        def extract_data_from_series(series, regex, dtype = "float"): 
            """
            Extracts a single data point from a series of strings. Assumes only one match 
            will exist, so only first observation will be returned. 
            """
            return (series.loc[series.str.match(regex, case = False)]
                       .str.extract(regex)
                       .values[0].astype(dtype))
        
        global_score = pd.Series(data[0])
        other_scores = pd.Series(data[1])
        #make sure sustainbility data exists (not every fund has it)
        
        
        globe_rating_regex = r"Rating ([0-5]) Out of 5"
        try: 
            esg_dict["sustainability_rating"] = extract_data_from_series(other_scores, globe_rating_regex, "int")[0]
        except IndexError: 
            return esg_dict 
        
        #first series conatains information about global_category count and sustainable investment indicator (yes/no)
        yes_no_regex = r"no|yes"
        sustainable_investment_loc = global_score.loc[global_score.str.match(yes_no_regex, case = False)]
        esg_dict["sustainable_investment"] = sustainable_investment_loc.values[0]
        
        #gobal_count is always one before the sustainble_investment (y/n) category
        esg_dict["global_category_count"] = int(global_score.loc[sustainable_investment_loc.index - 1].values[0])

        #second series contains all other information
        
        environmental_rating_regex = r"Environmental ([0-9]{1,2}\.[0-9]{1,2})"
        social_rating_regex = r"Social ([0-9]{1,2}\.[0-9]{2})"
        governance_rating_regex = r"Governance ([0-9]{1,2}\.[0-9]{1,2})"
        unallocated_rating_regex = r"Unallocated ([0-9]{1,2}\.[0-9]{1,2})"

        sustainibility_score_regex = r"Historical score ([0-9]{1,2}\.[0-9]{1,2}) Out of Fifty, Current Score ([0-9]{1,2}\.[0-9]{1,2}) Out of Fifty, Historical Average ([0-9]{2}\.[0-9]{1,2}) Out of Fifty"
        carbon_risk_score_regex = r"Carbon Risk Score, ([0-9]{1,2}\.[0-9]{1,2})? ?Out Of Hundred\. Carbon Risk Score Category Low, ([0-9]{1,2}\.[0-9]{1,2}) Out Of Hundred\. Carbon Risk Score Category High, ([0-9]{1,2}\.[0-9]{1,2}) Out Of Hundred\. Carbon Risk Score Category Average, ([0-9]{1,2}\.[0-9]{1,2}) Out Of Hundred\."
        fossil_fuel_involvement_regex = r"Fossil Fuel Involvement %, ([0-9]{1,2}\.[0-9]{1,2})? ?Out Of Hundred\. Fossil Fuel Involvement % Category Low, ([0-9]{1,2}\.[0-9]{1,2}) Out Of Hundred\. Fossil Fuel Involvement % Category High, ([0-9]{1,3}\.[0-9]{1,2}) Out Of Hundred. Fossil Fuel Involvement % Category Average, ([0-9]{1,2}\.[0-9]{1,2}) Out Of Hundred."

    

        #extract ratings from series using regex's defined above
        try: 
            esg_dict["sustainability_rating"] = extract_data_from_series(other_scores, globe_rating_regex, "int")[0]
        except IndexError: 
            return esg_dict 
        
        esg_dict["hist_sustainability_score"], esg_dict["current_sus_score"], esg_dict["hist_avg"] = extract_data_from_series(other_scores, sustainibility_score_regex)
        esg_dict["environmental_rating"] = extract_data_from_series(other_scores, environmental_rating_regex)[0]
        esg_dict["social_rating"] = extract_data_from_series(other_scores, social_rating_regex)[0]
        esg_dict["governance_rating"] = extract_data_from_series(other_scores, governance_rating_regex)[0]
        esg_dict["unallocated_rating"] = extract_data_from_series(other_scores, unallocated_rating_regex)[0]
        esg_dict["carbon_current"], esg_dict["carbon_low"], esg_dict["carbon_high"], esg_dict["carbon_average"] = extract_data_from_series(other_scores, carbon_risk_score_regex)
        esg_dict["fossil_current"], esg_dict["fossil_low"], esg_dict["fossil_high"], esg_dict["fossil_avg"] = extract_data_from_series(other_scores, fossil_fuel_involvement_regex)

        return esg_dict
    
    def close_parser(self): 
        """
        Ends Selenium session, and shuts down the parser. 
        Parser will not function properly after a call to close_parser, please 
        start a new parser, and don't call this method until parsing is complete. 
        """
        self.driver.close()
        

In [191]:
parser = ESG_Parser()

C:\Users\Ayush\AppData\Local\Temp/ipykernel_61256/2256197907.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver_win32/chromedriver.exe")


In [192]:
for i in range(20, 100):
    test_link = funds.link.iloc[i]
    data = parser.get_data_from_webpage(test_link)
    structured = parser.parse_data(data) 
    print(structured)

{'sustainability_rating': None, 'global_category_count': None, 'sustainable_investment': None, 'hist_sustainability_score': None, 'current_sus_score': None, 'hist_avg': None, 'environmental_rating': None, 'social_rating': None, 'governance_rating': None, 'unallocated_rating': None, 'carbon_current': None, 'carbon_low': None, 'carbon_high': None, 'carbon_average': None, 'fossil_current': None, 'fossil_low': None, 'fossil_high': None, 'fossil_avg': None}
{'sustainability_rating': 3, 'global_category_count': 4915, 'sustainable_investment': 'No', 'hist_sustainability_score': 22.8, 'current_sus_score': 22.49, 'hist_avg': 24.29, 'environmental_rating': 4.09, 'social_rating': 8.68, 'governance_rating': 6.56, 'unallocated_rating': 3.15, 'carbon_current': 8.4, 'carbon_low': 2.42, 'carbon_high': 22.57, 'carbon_average': 8.69, 'fossil_current': 5.46, 'fossil_low': 0.0, 'fossil_high': 40.99, 'fossil_avg': 8.95}
{'sustainability_rating': 4, 'global_category_count': 3631, 'sustainable_investment': '

{'sustainability_rating': None, 'global_category_count': None, 'sustainable_investment': None, 'hist_sustainability_score': None, 'current_sus_score': None, 'hist_avg': None, 'environmental_rating': None, 'social_rating': None, 'governance_rating': None, 'unallocated_rating': None, 'carbon_current': None, 'carbon_low': None, 'carbon_high': None, 'carbon_average': None, 'fossil_current': None, 'fossil_low': None, 'fossil_high': None, 'fossil_avg': None}
{'sustainability_rating': None, 'global_category_count': None, 'sustainable_investment': None, 'hist_sustainability_score': None, 'current_sus_score': None, 'hist_avg': None, 'environmental_rating': None, 'social_rating': None, 'governance_rating': None, 'unallocated_rating': None, 'carbon_current': None, 'carbon_low': None, 'carbon_high': None, 'carbon_average': None, 'fossil_current': None, 'fossil_low': None, 'fossil_high': None, 'fossil_avg': None}
{'sustainability_rating': None, 'global_category_count': None, 'sustainable_investment

{'sustainability_rating': None, 'global_category_count': None, 'sustainable_investment': None, 'hist_sustainability_score': None, 'current_sus_score': None, 'hist_avg': None, 'environmental_rating': None, 'social_rating': None, 'governance_rating': None, 'unallocated_rating': None, 'carbon_current': None, 'carbon_low': None, 'carbon_high': None, 'carbon_average': None, 'fossil_current': None, 'fossil_low': None, 'fossil_high': None, 'fossil_avg': None}
{'sustainability_rating': 4, 'global_category_count': 2043, 'sustainable_investment': 'No', 'hist_sustainability_score': 26.55, 'current_sus_score': 26.59, 'hist_avg': 30.05, 'environmental_rating': 3.29, 'social_rating': 11.2, 'governance_rating': 8.29, 'unallocated_rating': 3.81, 'carbon_current': 5.84, 'carbon_low': 1.41, 'carbon_high': 41.64, 'carbon_average': 10.82, 'fossil_current': 0.25, 'fossil_low': 0.0, 'fossil_high': 94.09, 'fossil_avg': 4.01}
{'sustainability_rating': None, 'global_category_count': None, 'sustainable_investme

{'sustainability_rating': 1, 'global_category_count': 1761, 'sustainable_investment': 'No', 'hist_sustainability_score': 23.77, 'current_sus_score': 23.61, 'hist_avg': 22.77, 'environmental_rating': 4.14, 'social_rating': 8.92, 'governance_rating': 6.76, 'unallocated_rating': 3.79, 'carbon_current': nan, 'carbon_low': 6.18, 'carbon_high': 11.39, 'carbon_average': 7.89, 'fossil_current': nan, 'fossil_low': 0.8, 'fossil_high': 19.81, 'fossil_avg': 6.26}
{'sustainability_rating': None, 'global_category_count': None, 'sustainable_investment': None, 'hist_sustainability_score': None, 'current_sus_score': None, 'hist_avg': None, 'environmental_rating': None, 'social_rating': None, 'governance_rating': None, 'unallocated_rating': None, 'carbon_current': None, 'carbon_low': None, 'carbon_high': None, 'carbon_average': None, 'fossil_current': None, 'fossil_low': None, 'fossil_high': None, 'fossil_avg': None}
{'sustainability_rating': 1, 'global_category_count': 1761, 'sustainable_investment': '

{'sustainability_rating': 2, 'global_category_count': 1761, 'sustainable_investment': 'No', 'hist_sustainability_score': 23.54, 'current_sus_score': 23.39, 'hist_avg': 22.77, 'environmental_rating': 4.06, 'social_rating': 9.0, 'governance_rating': 6.86, 'unallocated_rating': 3.48, 'carbon_current': nan, 'carbon_low': 6.2, 'carbon_high': 13.9, 'carbon_average': 7.9, 'fossil_current': nan, 'fossil_low': 0.69, 'fossil_high': 21.51, 'fossil_avg': 6.35}
{'sustainability_rating': 2, 'global_category_count': 1761, 'sustainable_investment': 'No', 'hist_sustainability_score': 23.54, 'current_sus_score': 23.39, 'hist_avg': 22.77, 'environmental_rating': 4.06, 'social_rating': 9.0, 'governance_rating': 6.86, 'unallocated_rating': 3.48, 'carbon_current': nan, 'carbon_low': 6.2, 'carbon_high': 13.9, 'carbon_average': 7.9, 'fossil_current': nan, 'fossil_low': 0.69, 'fossil_high': 21.51, 'fossil_avg': 6.35}
{'sustainability_rating': 1, 'global_category_count': 7083, 'sustainable_investment': 'No', 'h

IndexError: single positional indexer is out-of-bounds

In [219]:
funds.loc[1].link

'https://www.morningstar.com/funds/xnas/aaaax/portfolio'

In [140]:
carbon_risk_score_regex = r"Carbon Risk Score, ([0-9]{1,2}\.[0-9]{1,2})? ?Out Of Hundred\. Carbon Risk Score Category Low, ([0-9]{1,2}\.[0-9]{1,2}) Out Of Hundred\. Carbon Risk Score Category High, ([0-9]{1,2}\.[0-9]{1,2}) Out Of Hundred\. Carbon Risk Score Category Average, ([0-9]{1,2}\.[0-9]{1,2}) Out Of Hundred\."
sample_string = "Carbon Risk Score, Out Of Hundred. Carbon Risk Score Category Low, 3.30 Out Of Hundred. Carbon Risk Score Category High, 15.81 Out Of Hundred. Carbon Risk Score Category Average, 9.13 Out Of Hundred.'"

series = pd.Series(sample_string)
series.str.extract(carbon_risk_score_regex)[0]



0    NaN
Name: 0, dtype: object

Items to retrieve 
- Sustainbility Rating (number of globes) 
- Historical sustainibility score rank 
- Number of funds in global category 
- sustainable investment(yes, no) 
- Carbon Risk Score 
- Fossil Fuel Involvment


In [190]:
parser.close_parser()

In [147]:
driver.close()